# Predicting Pennsylvania with babypandas

In [1]:
import babypandas as bpd
import numpy as np

## Who will win?

As of Thursday afternoon on November 5, Donald Trump is leading Joe Biden in Pennsylvania by over 100,000 votes, with about 340,000 votes left to count. If these votes were a true random sample from the population of Pennsylvania, Biden would have no real chance of catching up. However, these votes are from *mail-in* ballots, which are known to overwhelmingly break towards Biden. Can he catch Trump, win Pennsylvania, and therefore win the election? Let's see.

In [2]:
# Trump's current lead in PA
CURRENT_GAP = 108_367

## The plan

Here's the idea. Pennsylvania publishes current election data on [data.pa.gov](http://data.pa.gov). In particular, they [publish a data set](https://data.pa.gov/Government-Efficiency-Citizen-Engagement/2020-General-Election-Unofficial-Mail-Ballot-Proce/pg3c-9a9m) of the number of mail-in ballots that remain to be counted on a per-county basis. Let's read it in here:

In [3]:
remaining_raw = bpd.read_csv('data/2020_General_Election_Unofficial_Mail_Ballot_Processing_Current_Hourly_County_State.csv')
remaining_raw

,Ballots Issued to Voters,Ballots Cast,County,% counted,Ballots Counted,Ballots Remaining
0,19460,15046,MERCER,0.3693,5556,9490
1,11269,9299,NORTHUMBERLAND,0.9824,9135,164
2,64958,55087,DAUPHIN,0.9836,54186,901
3,16166,12679,LAWRENCE,0.9437,11965,714
4,5708,4930,MIFFLIN,0.8696,4287,643
...,...,...,...,...,...,...
62,5464,4593,TIOGA,0.0000,0,4593
63,6886,6130,UNION,0.9737,5969,161
64,5162,4616,HUNTINGDON,0.9913,4576,40
65,1538,1253,FULTON,0.9649,1209,44


Biden does well with main in ballots. Let's do the simplest possible analysis: suppose Biden wins 66% of the mail in ballots. How many *more* votes will he win than Trump?

In [4]:
simple_for_biden = (remaining_raw.get('Ballots Remaining') * .66).sum()
simple_for_trump = (remaining_raw.get('Ballots Remaining') * .33).sum()

In [5]:
simple_for_biden - simple_for_trump

112323.75

This isn't enough to catch up (the current gap is larger), but it is very close!

Now, we know that some of these counties lean toward Trump and some lean towards Biden. If most of the remaining votes are in Biden-leaning counties, he has a better chance of catching up. To start, we can use the [by county](https://www.nytimes.com/interactive/2020/11/03/us/elections/results-pennsylvania.html) results from the New York Times to get a sense of how each county votes:

In [6]:
nyt_margins_raw = bpd.read_csv('data/nyt_margin.csv').set_index('County')
nyt_margins_raw

,Winner,Point Margin
County,,
CRAWFORD,Trump,59.0
CUMBERLAND,Trump,25.0
MERCER,Trump,38.0
LEHIGH,Trump,0.4
TIOGA,Trump,66.0
...,...,...
POTTER,Trump,61.0
FULTON,Trump,72.0
SULLIVAN,Trump,48.0


This shows who has won the county with all of the votes counted so far, and the margin of their victory in "points". If Trump gets 53% of the vote in a county, and Biden receives 47%, Trump has won by 6%, or 6 points.

We could use these percentages to predict how many of the remaining votes will go for Biden, but we'd be undercounting. Why? These percentages are for mostly in-person votes, which favor Trump. Take for instance Crawford County.

In [7]:
nyt_margins_raw.loc['CRAWFORD']

Winner          Trump
Point Margin       59
Name: CRAWFORD, dtype: object

Trump won the in-person vote in Crawford by almost 60 points! That means he got 80% of the vote, to Biden's 20%.
But Biden won *63%* of the mail-in votes, according to [538](https://fivethirtyeight.com/live-blog/2020-election-results-coverage/). This is a huge difference! This pattern repeats itself all over the state. Biden had 17% of the in-person vote in Tioga county, but 50% of the mail-in vote, for instance.

All over, it looks like Biden gets a "mail-in boost" of about 30 - 40%. That is to say, we might predict that the percentage of mail-in ballots won by Biden is, say, the in-person percentage won by Biden, plus 35%. This is the basic idea of the approach.

Of course, in areas like Philadelphia, Biden was winning a huge amount (75%+) of the in-person vote. Adding 35% to this gives 110%! That's not good. Clearly, there is less room for Biden to improve here, so we'll say that in Biden-leaning counties, Biden will be boosted less -- by maybe 5% to 10%.

So here's the model: if county was leaning for Trump before mail-in votes are counted, the mail-in ballots will lean towards Biden heavily. Think: Tioga county. We'll make this a parameter, but we'll say about 35%.

If a county was leaning towards Biden on election day, the boost he gets from mail-in is less. We'll start at 35% for counties that are 50/50 Biden/Trump, and taper it down linearly to 0% for counties that voted entirely for Biden. Here's a function which does this.

In [8]:
def predicted_percent_biden(in_person_pct, baseline_boost):
    """baseline_boost is the boost for red counties; something like 35%"""
    if in_person_pct <= .5:
        return baseline_boost + in_person_pct
    else:
        return 2*(1 - in_person_pct)*baseline_boost + in_person_pct

Let's test it out. Here is the boost for a county that went 30% Biden in-person:

In [9]:
predicted_percent_biden(.30, .35)

0.6499999999999999

This says that the mail-in vote would be roughly 65% Biden.

Here's the boost for a county that went 75% Biden:

In [10]:
predicted_percent_biden(.75, .35)

0.925

This says that the mail-in vote for Biden would be >90%. Is this too high? We can come back to this later, but it isn't too far from what has been observed (>80%).

## Data cleaning

We're going to need to join our two data sets, so let's make sure that both tables have their county names in all capital letters and get rid of any extra stuff.

In [11]:
nyt_margins = (
    nyt_margins_raw
    .assign(UpperCounty=nyt_margins_raw.index.str.upper())
    .reset_index()
    .get(['UpperCounty', 'Point Margin', 'Winner'])
    .set_index('UpperCounty')
)
nyt_margins

,Point Margin,Winner
UpperCounty,,
CRAWFORD,59.0,Trump
CUMBERLAND,25.0,Trump
MERCER,38.0,Trump
LEHIGH,0.4,Trump
TIOGA,66.0,Trump
...,...,...
POTTER,61.0,Trump
FULTON,72.0,Trump
SULLIVAN,48.0,Trump


In [12]:
remaining = remaining_raw.set_index('County').get(['Ballots Remaining'])
remaining

,Ballots Remaining
County,
MERCER,9490
NORTHUMBERLAND,164
DAUPHIN,901
LAWRENCE,714
MIFFLIN,643
...,...
TIOGA,4593
UNION,161
HUNTINGDON,40


Now we'll merge:

In [13]:
merged = remaining.merge(nyt_margins, left_index=True, right_index=True)
merged

,Ballots Remaining,Point Margin,Winner
MERCER,9490,38.0,Trump
NORTHUMBERLAND,164,38.0,Trump
DAUPHIN,901,8.0,Biden
LAWRENCE,714,30.0,Trump
MIFFLIN,643,56.0,Trump
...,...,...,...
TIOGA,4593,66.0,Trump
UNION,161,25.0,Trump
HUNTINGDON,40,51.0,Trump
FULTON,44,72.0,Trump




Let's add a column to the table which contains the percentage of the votes earned by Biden. We'll start by making the point margin negative if Trump won the county, and positive if Biden won.

In [14]:
def margin_sign(s):
    if s == 'Trump':
        return -1
    else:
        return 1

with_signed_margins = merged.assign(
    signed_margin=merged.get('Point Margin') * merged.get('Winner').apply(margin_sign)
)
with_signed_margins

,Ballots Remaining,Point Margin,Winner,signed_margin
MERCER,9490,38.0,Trump,-38.0
NORTHUMBERLAND,164,38.0,Trump,-38.0
DAUPHIN,901,8.0,Biden,8.0
LAWRENCE,714,30.0,Trump,-30.0
MIFFLIN,643,56.0,Trump,-56.0
...,...,...,...,...
TIOGA,4593,66.0,Trump,-66.0
UNION,161,25.0,Trump,-25.0
HUNTINGDON,40,51.0,Trump,-51.0
FULTON,44,72.0,Trump,-72.0


Now let's convert the points to percentage:

In [15]:
with_in_person_pct = with_signed_margins.assign(
    in_person_biden=(with_signed_margins.get('signed_margin') / 2 + 50) / 100
)
with_in_person_pct

,Ballots Remaining,Point Margin,Winner,signed_margin,in_person_biden
MERCER,9490,38.0,Trump,-38.0,0.310
NORTHUMBERLAND,164,38.0,Trump,-38.0,0.310
DAUPHIN,901,8.0,Biden,8.0,0.540
LAWRENCE,714,30.0,Trump,-30.0,0.350
MIFFLIN,643,56.0,Trump,-56.0,0.220
...,...,...,...,...,...
TIOGA,4593,66.0,Trump,-66.0,0.170
UNION,161,25.0,Trump,-25.0,0.375
HUNTINGDON,40,51.0,Trump,-51.0,0.245
FULTON,44,72.0,Trump,-72.0,0.140


Now let's get our predicted percent for mail-in for Biden, using a 35% mail-in boost for Biden in red counties, tapering to 0% for fully-blue counties.

In [16]:
def predicted_biden_35(p):
    """Boost Biden by 35% in red counties"""
    return predicted_percent_biden(p, .35)
    

with_predicted_pct = with_in_person_pct.assign(
    predicted_biden_pct=with_in_person_pct.get('in_person_biden').apply(predicted_biden_35)
)
with_predicted_pct

,Ballots Remaining,Point Margin,Winner,signed_margin,in_person_biden,predicted_biden_pct
MERCER,9490,38.0,Trump,-38.0,0.310,0.660
NORTHUMBERLAND,164,38.0,Trump,-38.0,0.310,0.660
DAUPHIN,901,8.0,Biden,8.0,0.540,0.862
LAWRENCE,714,30.0,Trump,-30.0,0.350,0.700
MIFFLIN,643,56.0,Trump,-56.0,0.220,0.570
...,...,...,...,...,...,...
TIOGA,4593,66.0,Trump,-66.0,0.170,0.520
UNION,161,25.0,Trump,-25.0,0.375,0.725
HUNTINGDON,40,51.0,Trump,-51.0,0.245,0.595
FULTON,44,72.0,Trump,-72.0,0.140,0.490


## Results

Alright, now for the results. How many votes will Biden pick up from mail-in votes under our model?

In [17]:
biden_35 = (with_predicted_pct.get('predicted_biden_pct') * with_predicted_pct.get('Ballots Remaining')).sum()
biden_35

280386.39149999997

And Trump?

In [18]:
trump_35 = ((1 - with_predicted_pct.get('predicted_biden_pct')) * with_predicted_pct.get('Ballots Remaining')).sum()
trump_35

59988.60850000001

This means that Biden picks up this many more votes than Trump:

In [19]:
biden_35 - trump_35

220397.78299999997

And therefore wins PA by:

In [20]:
(biden_35 - trump_35) - CURRENT_GAP

112030.78299999997

## Is this realistic?

Our boost to Biden in blue counties might be too nice, as it says that he might win >90% of ballots. Let's put a cap of 85% on the percentage of ballots won by Biden in any county.

In [21]:
def predicted_biden(p, alpha):
    """Boost Biden by alpha% in red counties, but cap percentage at .85"""
    return np.minimum(predicted_percent_biden(p, alpha), .85)

Is 35% a reasonable boost? The data suggests it is, but let's be conservative and try 20%. In fact, let's turn the analysis into a function so we can test several boosts:

In [22]:
def predict_pennsylvania(baseline_boost):
    
    def mail_in_boost(p):
        return predicted_biden(p, baseline_boost)
    
    with_predicted_pct = with_in_person_pct.assign(
        predicted_biden_pct=with_in_person_pct.get('in_person_biden').apply(mail_in_boost)
    )
    
    biden_gain = (with_predicted_pct.get('predicted_biden_pct') * with_predicted_pct.get('Ballots Remaining')).sum()
    trump_gain = ((1 - with_predicted_pct.get('predicted_biden_pct')) * with_predicted_pct.get('Ballots Remaining')).sum()
    
    return biden_gain, trump_gain

In [26]:
# change this to whatever you'd like
# the race gets really close at baseline_boost = .15
baseline_boost = 0.35

biden_gain, trump_gain = predict_pennsylvania(baseline_boost)
difference = biden_gain - trump_gain
outcome = difference - CURRENT_GAP

print(f"Assuming Biden does {baseline_boost * 100}% better on mail-in as compared to in-person in red counties.")
print(f"And assuming that Biden's mail-in boost is less as the county gets bluer.")
print(f'And assuming that Biden earns no more than 85% of mail-in votes in any county.')
print(f'Biden is projected to {"win" if outcome > 0 else "lose"} PA by {int(outcome)} votes.')

Assuming Biden does 35.0% better on mail-in as compared to in-person in red counties.
And assuming that Biden's mail-in boost is less as the county gets bluer.
And assuming that Biden earns no more than 85% of mail-in votes in any county.
Biden is projected to win PA by 91851 votes.


## Conclusion

Biden is projected to win PA by around 80,000 - 120,000 votes in most plausible scenarios. However, I'm not Nate Silver. Even if this model is good, it is contingent upon the number of remaining votes in each county being accurate -- and that isn't always the case! Take it with a grain of salt.